In [1]:
%cd ..

E:\システムトレード入門\trade_system_git_workspace


In [68]:
from pathlib import Path
import datetime
from pytz import timezone
import pandas as pd
import numpy as np

In [2]:
from get_stock_price import StockDatabase

In [4]:
db_path = Path("db/stock_db") / Path("stock.db")
stock_db = StockDatabase(db_path)

### 利用するDataFrame

In [79]:
jst_timezone = timezone("Asia/Tokyo")
start_datetime = jst_timezone.localize(datetime.datetime(2020,11,5,9,0,0))
end_datetime = jst_timezone.localize(datetime.datetime(2020,12,1,15,0,0))

stock_df = stock_db.search_span(stock_names=["6502"], 
                                start_datetime=start_datetime,
                                end_datetime=end_datetime,
                                freq_str="T",
                                to_tokyo=True,
                               )
stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### datetime.datetimeによるインデキシング 

上のデータはdatetimeindexがawareなdatetimeなので，インデキシングに利用するdatetimeもawareでなければならない．`pd.DatetimeIndex`(あるいは他のindexでも)の比較はbooleanのndarrayが返る．

In [115]:
bool_array = stock_df.index > jst_timezone.localize(datetime.datetime(2020,11,16,9,0,0))
sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-16 09:01:00+09:00,2708.0,2711.0,2708.0,2708.0,7800.0
2020-11-16 09:02:00+09:00,2708.0,2708.0,2701.0,2702.0,14900.0
2020-11-16 09:03:00+09:00,2703.0,2712.0,2701.0,2703.0,32600.0
2020-11-16 09:04:00+09:00,2703.0,2706.0,2701.0,2702.0,12700.0
2020-11-16 09:05:00+09:00,2703.0,2706.0,2698.0,2705.0,57400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0


### 範囲を指定してインデキシング 

もちろんdatetime.dateを指定してもエラーが出る

In [116]:
sub_stock_df = stock_df.loc[datetime.date(2020,12,1)]
sub_stock_df

KeyError: datetime.date(2020, 12, 1)

期間指定する場合は素直にboolインデックスを利用する.

In [117]:
bool_array = (jst_timezone.localize(datetime.datetime(2020,11,16,9,0,0)) <= stock_df.index) & (stock_df.index < jst_timezone.localize(datetime.datetime(2020,11,16,15,0,0)))
sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0
2020-11-16 09:01:00+09:00,2708.0,2711.0,2708.0,2708.0,7800.0
2020-11-16 09:02:00+09:00,2708.0,2708.0,2701.0,2702.0,14900.0
2020-11-16 09:03:00+09:00,2703.0,2712.0,2701.0,2703.0,32600.0
2020-11-16 09:04:00+09:00,2703.0,2706.0,2701.0,2702.0,12700.0
...,...,...,...,...,...
2020-11-16 14:55:00+09:00,2715.0,2716.0,2715.0,2715.0,6400.0
2020-11-16 14:56:00+09:00,2716.0,2716.0,2714.0,2716.0,25500.0
2020-11-16 14:57:00+09:00,2715.0,2717.0,2715.0,2715.0,13800.0


複数期間の場合は，普通にbooleanのarrayを論理和(&)すればよい

### 複数の特定のインデックスを取り出す

これはDatetimeIndexに限ったことではないが，pd.DatatetimeIndex.isinメソッドを利用できる．

In [112]:
datetime_list = [jst_timezone.localize(datetime.datetime(2020,11,16,9,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,10,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,11,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,13,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,14,0,0)),
                ]

bool_array = stock_df.index.isin(datetime_list)
sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0
2020-11-16 10:00:00+09:00,2710.0,2714.0,2710.0,2713.0,9400.0
2020-11-16 11:00:00+09:00,2715.0,2716.0,2715.0,2715.0,1100.0
2020-11-16 13:00:00+09:00,2706.0,2706.0,2705.0,2705.0,500.0
2020-11-16 14:00:00+09:00,2717.0,2717.0,2716.0,2716.0,5000.0


### 複数の特定の日時のものを取り出す 

ある日にちもののみ取り出すときに，その日にちが複数ある場合．`pd.DatetimeIndex.date`によって`datetime.date`のndarrayを返し，numpyの`np.isin`か`np.in1d`をとる．このとき`datetime.date`になるので，タイムゾーンとかは無視される．

In [142]:
date_list = [datetime.date(2020, 11, 10),
             datetime.date(2020, 11, 11),
             datetime.date(2020, 11, 16),
             datetime.date(2020, 11, 17)
            ]
date_array = np.array(date_list)
bool_array = np.in1d(stock_df.index.date, date_array)

sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-10 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:01:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:02:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:03:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:04:00+09:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-17 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


### 特定の曜日のデータを取得

pd.DatetimeIndex.weekdayを指定することでブールインデックスを取得できる．0が月曜日で6が日曜日(datetime.datetime.weekdayと同じ)

In [105]:
bool_array = stock_df.index.weekday==0
stock_df.loc[bool_array]

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-09 09:01:00+09:00,2711.0,2712.0,2701.0,2706.0,12900.0
2020-11-09 09:02:00+09:00,2705.0,2710.0,2701.0,2701.0,9200.0
2020-11-09 09:03:00+09:00,2702.0,2703.0,2692.0,2694.0,19300.0
2020-11-09 09:04:00+09:00,2696.0,2698.0,2686.0,2688.0,10800.0
...,...,...,...,...,...
2020-11-30 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


### 毎日のある時刻を取得 

#### 一つのデータを取得する場合

datetime.timeオブジェクトをインデックスに指定することで，毎日のその時刻のデータを取得できる．これはdatetimeindexがawareなdatetimeの値でもよい．
注意しなければいけないのは，以下では9時00分00秒のデータを取得していること．これは`datetime.time(9)`としても同じ，もちろんリサンプリング等はされない．

In [82]:
stock_df.loc[datetime.time(9,0)].head(5)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN


実は，これは`pandas.DataFrame.at_time`を適用するのと等価である

In [83]:
stock_df.at_time(datetime.time(9,0)).head(5)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN


**コラム**  
このデータベースではyahoofinanceのapiを利用してデータを修正しているが，データの始まり・終わりが特徴的である．以下では取引時間の終わりと始まりのデータを集めてみる．すると，11時30分以降の1分間のデータが取得できていることが分かる．

In [84]:
print("at  9:00 am:\n",stock_df.at_time(datetime.time(9,0)).head(5))
print("at 11:30 am:\n",stock_df.at_time(datetime.time(11,30)).head(5))
print("at 12:30 am:\n",stock_df.at_time(datetime.time(12,30)).head(5))
print("at 15:00 am:\n",stock_df.at_time(datetime.time(15,0)).head(5))

at  9:00 am:
                            Open_6502  High_6502  Low_6502  Close_6502  Volume_6502
timestamp                                                                         
2020-11-05 09:00:00+09:00     2631.0     2641.0    2630.0      2639.0      55000.0
2020-11-06 09:00:00+09:00     2615.0     2644.0    2609.0      2617.0      90700.0
2020-11-09 09:00:00+09:00     2711.0     2719.0    2707.0      2710.0          0.0
2020-11-10 09:00:00+09:00     2780.0     2782.0    2770.0      2777.0     142700.0
2020-11-11 09:00:00+09:00        NaN        NaN       NaN         NaN          NaN
at 11:30 am:
                            Open_6502  High_6502  Low_6502  Close_6502  Volume_6502
timestamp                                                                         
2020-11-05 11:30:00+09:00     2640.0     2640.0    2640.0      2640.0       1000.0
2020-11-06 11:30:00+09:00     2649.0     2649.0    2649.0      2649.0        400.0
2020-11-09 11:30:00+09:00     2735.0     2735.0    2735.0  

開始時刻の一分前と終了時刻の一分後は

In [85]:
print("at  8:59 am:\n",stock_df.at_time(datetime.time(8,59)).head(5))
print("at 11:31 am:\n",stock_df.at_time(datetime.time(11,31)).head(5))
print("at 12:29 am:\n",stock_df.at_time(datetime.time(12,29)).head(5))
print("at 15:01 am:\n",stock_df.at_time(datetime.time(15,1)).head(5))

at  8:59 am:
                            Open_6502  High_6502  Low_6502  Close_6502  Volume_6502
timestamp                                                                         
2020-11-06 08:59:00+09:00        NaN        NaN       NaN         NaN          NaN
2020-11-10 08:59:00+09:00        NaN        NaN       NaN         NaN          NaN
2020-11-11 08:59:00+09:00        NaN        NaN       NaN         NaN          NaN
2020-11-12 08:59:00+09:00        NaN        NaN       NaN         NaN          NaN
2020-11-13 08:59:00+09:00        NaN        NaN       NaN         NaN          NaN
at 11:31 am:
                            Open_6502  High_6502  Low_6502  Close_6502  Volume_6502
timestamp                                                                         
2020-11-05 11:31:00+09:00        NaN        NaN       NaN         NaN          NaN
2020-11-06 11:31:00+09:00        NaN        NaN       NaN         NaN          NaN
2020-11-09 11:31:00+09:00        NaN        NaN       NaN  

ここから，yahoofinanceで得られるデータについて，開始時刻はぴったり始まり，一分前のデータを取得できない．しかし終了時刻については，15時00分はぴったり終了し，一分後のデータが取れないが，11時30分の一分間はデータが取得されることが分かる．

#### 範囲指定 

datetime.timeをスライスのように利用することで，毎日のある時刻から時刻までを取得できる．しかし，これは通常のスライスと異なり，最後を含んでしまうことに注意する．これはデータを少しづつとるときにデータの重複を起こしてしまう．

In [87]:
stock_df.loc[datetime.time(9,0):datetime.time(11,30)]

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 11:26:00+09:00,2942.0,2942.0,2939.0,2939.0,3800.0
2020-12-01 11:27:00+09:00,2939.0,2939.0,2936.0,2939.0,4700.0
2020-12-01 11:28:00+09:00,2940.0,2942.0,2940.0,2942.0,2700.0


そこで，対応するメソッドである`pandas.Dataframe.between_time`を利用し，`include_end`を`False`と指定することで一般的なスライスと同等にできる．

In [88]:
stock_df.between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 11:25:00+09:00,2944.0,2944.0,2942.0,2942.0,3100.0
2020-12-01 11:26:00+09:00,2942.0,2942.0,2939.0,2939.0,3800.0
2020-12-01 11:27:00+09:00,2939.0,2939.0,2936.0,2939.0,4700.0


#### 複数の条件を範囲で指定する場合

`pandas.DatetimeIndex.indexer_between_time`を使って対応するDatetimeIndexのインデックスが取得できる．

In [89]:
am_indice = stock_df.index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
am_indice

array([    0,     1,     2, ..., 28227, 28228, 28229], dtype=int64)

これを複数組み合わせることが考えられるが，インデックスを組み合わせた場合，その順序によって最終的に取得できる順番も変わってしまう．そのため，インデックスをそのまま組み合わせた後に，順番を並び替える必要がある．そこで，ブールインデックスとして処理することを考える．

In [90]:
bool_array = np.array([False]*len(stock_df))
am_indice = stock_df.index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
bool_array[am_indice] = True
pm_indice = stock_df.index.indexer_between_time(start_time=datetime.time(12,30), end_time=datetime.time(15,0), include_end=False)
bool_array[pm_indice] = True

intraday_stock_indice = stock_df.index[bool_array]
intraday_stock_indice

DatetimeIndex(['2020-11-05 09:00:00+09:00', '2020-11-05 09:01:00+09:00',
               '2020-11-05 09:02:00+09:00', '2020-11-05 09:03:00+09:00',
               '2020-11-05 09:04:00+09:00', '2020-11-05 09:05:00+09:00',
               '2020-11-05 09:06:00+09:00', '2020-11-05 09:07:00+09:00',
               '2020-11-05 09:08:00+09:00', '2020-11-05 09:09:00+09:00',
               ...
               '2020-12-01 14:50:00+09:00', '2020-12-01 14:51:00+09:00',
               '2020-12-01 14:52:00+09:00', '2020-12-01 14:53:00+09:00',
               '2020-12-01 14:54:00+09:00', '2020-12-01 14:55:00+09:00',
               '2020-12-01 14:56:00+09:00', '2020-12-01 14:57:00+09:00',
               '2020-12-01 14:58:00+09:00', '2020-12-01 14:59:00+09:00'],
              dtype='datetime64[ns, Asia/Tokyo]', name='timestamp', length=6600, freq=None)

In [91]:
intraday_stock_df = stock_df.loc[intraday_stock_indice,:]
intraday_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-05 09:01:00+09:00,2639.0,2646.0,2639.0,2644.0,5900.0
2020-11-05 09:02:00+09:00,2645.0,2648.0,2645.0,2646.0,4100.0
2020-11-05 09:03:00+09:00,2646.0,2648.0,2645.0,2645.0,4500.0
2020-11-05 09:04:00+09:00,2645.0,2646.0,2643.0,2643.0,2400.0
...,...,...,...,...,...
2020-12-01 14:55:00+09:00,2950.0,2952.0,2949.0,2951.0,25100.0
2020-12-01 14:56:00+09:00,2950.0,2951.0,2948.0,2949.0,16400.0
2020-12-01 14:57:00+09:00,2950.0,2955.0,2950.0,2954.0,25800.0
